In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import optuna
import pandas as pd

In [18]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

file_path = "/workspaces/codespaces-jupyter/data/cardio_train1.numbers"

# Confirm the file exists
assert os.path.exists(file_path), "CSV file not found. Upload the correct file."

# Read CSV
df = pd.read_csv(file_path, delimiter=';', encoding='utf-8', on_bad_lines='skip')

# Drop ID column if present
if 'id' in df.columns:
    df.drop('id', axis=1, inplace=True)

# Separate features and target
X = df.drop(columns=['cardio'])  # Target variable
y = df['cardio']

# SMOTE for class imbalance
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42)


NameError: name 'os' is not defined

In [ ]:

# Apply Polynomial Features (degree=3 for better feature interactions)
poly = PolynomialFeatures(degree=3, interaction_only=True)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# Hyperparameter Tuning using GridSearchCV for each model
param_grid_rf = {'n_estimators': [200, 300], 'max_depth': [15, 20], 'min_samples_split': [5, 10]}
param_grid_xgb = {'n_estimators': [300, 400], 'max_depth': [10, 15], 'learning_rate': [0.05, 0.1]}
param_grid_lgb = {'n_estimators': [250, 300], 'num_leaves': [31, 40], 'learning_rate': [0.05, 0.1]}

# Tune Random Forest
rf = RandomForestClassifier(random_state=42)
rf_best = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy')
rf_best.fit(X_train_scaled, y_train)

# Tune XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_best = GridSearchCV(xgb, param_grid_xgb, cv=5, scoring='accuracy')
xgb_best.fit(X_train_scaled, y_train)

# Tune LightGBM
lgb = LGBMClassifier(random_state=42)
lgb_best = GridSearchCV(lgb, param_grid_lgb, cv=5, scoring='accuracy')
lgb_best.fit(X_train_scaled, y_train)

# Best models after tuning
rf_final = rf_best.best_estimator_
xgb_final = xgb_best.best_estimator_
lgb_final = lgb_best.best_estimator_

# Decision Tree (less impact, so keeping default)
dt_final = DecisionTreeClassifier(max_depth=15, min_samples_split=10, random_state=42)
dt_final.fit(X_train_scaled, y_train)
# Individual model predictions and accuracy
# This code is moved from the separate cell to ensure variables are in scope
models = {'RandomForest': rf_final, 'XGBoost': xgb_final, 'LightGBM': lgb_final, 'DecisionTree': dt_final}
for model_name, model in models.items():
    y_pred_individual = model.predict(X_test_scaled)
    accuracy_individual = accuracy_score(y_test, y_pred_individual)
    print(f"📊 {model_name} Accuracy: {accuracy_individual * 100:.2f}%")


/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:20:42] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:20:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:21:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:21:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site

[LightGBM] [Info] Number of positive: 22349, number of negative: 22477
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024793 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36425
[LightGBM] [Info] Number of data points in the train set: 44826, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498572 -> initscore=-0.005711
[LightGBM] [Info] Start training from score -0.005711


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 22349, number of negative: 22477
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056750 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36437
[LightGBM] [Info] Number of data points in the train set: 44826, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498572 -> initscore=-0.005711
[LightGBM] [Info] Start training from score -0.005711


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 22349, number of negative: 22478
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.052319 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36333
[LightGBM] [Info] Number of data points in the train set: 44827, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498561 -> initscore=-0.005755
[LightGBM] [Info] Start training from score -0.005755


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 27936, number of negative: 28097
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.063390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 37014
[LightGBM] [Info] Number of data points in the train set: 56033, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498563 -> initscore=-0.005747
[LightGBM] [Info] Start training from score -0.005747
📊 RandomForest Accuracy: 74.00%
📊 XGBoost Accuracy: 73.97%
📊 LightGBM Accuracy: 74.30%
📊 DecisionTree Accuracy: 71.38%


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:

# ... (rest of the code, including stacking classifier and its accuracy) ...
# Define base models for stacking
estimators = [
    ('RandomForest', rf_final),  # Using the tuned Random Forest
    ('XGBoost', xgb_final),      # Using the tuned XGBoost
    ('LightGBM', lgb_final),    # Using the tuned LightGBM
    # You can add more base models here if needed
]

# Create the stacking classifier
# Using Logistic Regression as the final estimator (meta-learner)
stacking_classifier = StackingClassifier(
    estimators=estimators, 
    final_estimator=LogisticRegression(random_state=42) 
)

# Train the stacking classifier
stacking_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred_stacking = stacking_classifier.predict(X_test_scaled)

# Evaluate the stacking classifier's accuracy
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)
print(f"📊 Stacking Classifier Accuracy: {accuracy_stacking * 100:.2f}%")

# Define optimized stacking model
stacking_model = StackingClassifier(
    estimators=[
        ('RandomForest', rf_final),
        ('XGBoost', xgb_final),
        ('LightGBM', lgb_final),
        ('DecisionTree', dt_final)
    ],
    final_estimator=XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=7, random_state=42)
)

# Train the final stacking model
stacking_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = stacking_model.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"📊 Final Optimized Stacking Model Accuracy: {accuracy * 100:.2f}%")

# Classification report
print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred))

# Cross-validation score
cv_scores = cross_val_score(stacking_model, X_train_scaled, y_train, cv=15, scoring='accuracy')
print(f"\n✅ Mean Cross-Validation Accuracy: {np.mean(cv_scores) * 100:.2f}%")

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:44:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 27936, number of negative: 28097
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.064097 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 37014
[LightGBM] [Info] Number of data points in the train set: 56033, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498563 -> initscore=-0.005747
[LightGBM] [Info] Start training from score -0.005747


/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:51:32] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:51:49] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:52:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:52:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site

[LightGBM] [Info] Number of positive: 22349, number of negative: 22477
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050780 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36425
[LightGBM] [Info] Number of data points in the train set: 44826, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498572 -> initscore=-0.005711
[LightGBM] [Info] Start training from score -0.005711


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 20859, number of negative: 20980
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36189
[LightGBM] [Info] Number of data points in the train set: 41839, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498554 -> initscore=-0.005784
[LightGBM] [Info] Start training from score -0.005784


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



✅ Mean Cross-Validation Accuracy: 73.05%


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np

# Assuming stacking_model, scaler, and poly are already trained and available

# Define prediction function
def predict_cvd():
    try:
        # Get values from entry boxes
        inputs = [float(entry.get()) for entry in entries]
        
        # Convert to array and reshape
        X_input = np.array(inputs).reshape(1, -1)
        
        # Polynomial transformation and scaling
        X_input_poly = poly.transform(X_input)
        X_input_scaled = scaler.transform(X_input_poly)
        
        # Prediction
        prediction = stacking_model.predict(X_input_scaled)
        
        # Show result
        result = "You have CVD" if prediction[0] == 1 else "You don't have CVD"
        result_label.config(text=f"Prediction: {result}")
    except Exception as e:
        messagebox.showerror("Error", f"Invalid input: {e}")

# Tkinter GUI setup
root = tk.Tk()
root.title("CVD Prediction App")

labels = [
    "BMI (18.03–40)",
    "Age (30–65)",
    "Gender (1=Female, 2=Male)",
    "Cholesterol (1=Low, 2=Normal, 3=High)",
    "Glucose (1=Low, 2=Normal, 3=High)",
    "Smoke (0=No, 1=Yes)",
    "Alcohol (0=No, 1=Yes)",
    "Activity (0=No, 1=Yes)",
    "Avg_BP (50–190)"
]

entries = []

for i, label_text in enumerate(labels):
    tk.Label(root, text=label_text).grid(row=i, column=0, padx=5, pady=5, sticky="w")
    entry = tk.Entry(root)
    entry.grid(row=i, column=1, padx=5, pady=5)
    entries.append(entry)

# Predict button
tk.Button(root, text="Predict", command=predict_cvd).grid(row=len(labels), column=0, columnspan=2, pady=10)

# Prediction output
result_label = tk.Label(root, text="Prediction: ", font=('Arial', 12))
result_label.grid(row=len(labels) + 1, column=0, columnspan=2)

root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [ ]:
python cvd_gui.py


SyntaxError: invalid syntax (1058870826.py, line 1)